In [1]:
from magpie import grasp as gt
import rtde_control
import rtde_receive
from magpie.motor_code import Motors
from magpie import ur5 as ur5
import time
import numpy as np
import copy
import magpie
from magpie.gripper import Gripper

In [2]:
import importlib
importlib.reload(gt)
importlib.reload(ur5)
importlib.reload(magpie.gripper)

<module 'magpie.gripper' from '/home/will/MAGPIE/magpie/gripper.py'>

In [62]:
# set matrix
# tmat = np.array([[ 0.91574736, -0.35602909,  0.18614527, -0.02848649],
#        [-0.40073104, -0.77640133,  0.48643151,  0.14332736],
#        [ 0.02866033,  0.52004256,  0.85365937,  0.33750796],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]])

tmat = np.array([[ 0.98665647, -0.16180182, -0.01814324,  0.04438557],
       [-0.1565328 , -0.97333117,  0.16770185,  0.04194964],
       [ 0.04479385,  0.1626241 ,  0.98567079,  0.32025074],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

aperture = 37
# z_offset = G.aperture_to_z(aperture)/1000.0

mmc = copy.deepcopy(tmat[:3, 3])
# grasp_pose = [mmc[1], -mmc[0], mmc[2]]
grasp_pose = mmc
tmat[:3, 3] = [0, 0, 0]
homePose = None
z_offset = 100/1000.0
z_offset -= 0.035
z_offset

0.085

In [63]:
# move gripper to cartesian pos
# OR orient gripper in place
sleepRate = 1.5
# time.sleep(sleepRate)
ur = ur5.UR5_Interface()
try:
    ur.start()
    # homePose = ur.get_tcp_pose()
    homePose = ur.getPose()
    currentPose = ur.getPose()
    desiredPose = np.matmul(np.array(currentPose), tmat)
    print("Desired Pose: ", desiredPose)
    # in-place (or best attempt) re-orientation
    # ur.moveL(desiredPose)
    ## move to cartesian pos
    gt.move_to_L(grasp_pose, ur, z_offset=z_offset)    
    print("Done moving to block")
    time.sleep(sleepRate)
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Desired Pose:  [[ 0.127  0.956 -0.265 -0.247]
 [-0.992  0.128 -0.014 -0.089]
 [-0.021 -0.265 -0.964  0.204]
 [ 0.     0.     0.     1.   ]]
[[-0.025 -0.995 -0.1   -0.247]
 [-0.999  0.029 -0.037 -0.089]
 [ 0.04   0.099 -0.994  0.204]
 [ 0.     0.     0.     1.   ]]
P_goal:
[-0.066 -0.052 -0.228]
Done moving to block


In [64]:
# move home
ur = ur5.UR5_Interface()
try:
    ur.start()
    # time.sleep(sleepRate)
    # homePose = np.array([[-0.024, -0.998, -0.062, -0.261],
    #                     [-0.999,  0.021,  0.035, -0.162],
    #                     [-0.033,  0.063, -0.997,  0.221],
    #                     [ 0.   ,  0.   ,  0.   ,  1.   ]])
    ur.moveL(homePose)
    # gt_home = ur.getPose()
    # print(np.array(gt_home))
    time.sleep(sleepRate * 3)
    ur.stop()
except Exception as e:
    ur.stop()

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 


In [ ]:
# DO NOT EXECUTE

In [182]:
# # orient and move gripper to cartesian pos
# sleepRate = 1.5
# ur = ur5.UR5_Interface()
# try:
#     servoPort = "/dev/ttyACM0"
#     ur.start()
#     time.sleep(sleepRate)
#     homePose = ur.get_tcp_pose()    
#     '''
#     stopping point 2/15:
#     this works so far
#     take home pos
#     add translation on the gripper frame to the goal x, y
#     apply tmat to the goal
#     move to goal (translates on x, y and orients at the same time)
#     ## does this address the fact that orientation is not achievable in standstill?

#     then: take orig Z (from original gripper frame): [0, 0, dZ]
#     transform by tmat (the current orientation of the gripper)
#     add transformed dZ to the goal position
#     move
#     works!
#     '''
#     dX,dY,dZ = gt.get_world_frame(grasp_pose, ur, z_offset=z_offset)

#     goal1 = copy.deepcopy(homePose)
#     goal1 = np.array(goal1)
#     # goal1[:3, 3] += [dX, dY, 0]
#     goal1[:3, 3] += [dX, dY, dZ]
#     goal1 = np.array(goal1) @ tmat
#     ur.moveL(goal1)
#     time.sleep(sleepRate)

#     # todo: take currentPose and validate orientation?
#     goal2 = goal1
#     posd = [0, 0, dZ] @ tmat[:3, :3]
#     goal2 = np.array(goal2)
#     goal2[:3, 3] += posd
#     # ur.moveL(goal2)
#     time.sleep(sleepRate)
#     '''
#     end of working section
#     '''
#     ur.stop()
# except Exception as e:
#     ur.stop()
#     raise(e)